# Kickstarter 사이트의 펀딩 페이지의 content 및 risk and challenges 텍스트 마이닝
- 2015년~2019년 데이터만 수집
- 총 수집된 데이터 수/행 : 153,576 (608.5MB)

- 2015~2019년 각 연도별로 나누어 텍스트 데이터 수집 진행
- Kickstarter의 펀딩 페이지는 javascript를 통해 동적으로 텍스트 내용이 불러와지기에 Requests가 아닌 Selenium 사용
- 웹크롤링으로 데이터 수집 중 지속적으로 차단을 당하여 여러번의 수정을 거쳤다
    1. 오류가 발생하면 크롤링이 중단되고 그 중단 지점(index)이 csv에 저장되어 그 지점부터 manually 다시 크롤링을 시작
    2. 오류 발생시 크롤링을 중단하지 않고 계속 진행하되 오류가 난 지점들을 csv에 저장하고, 각 연도에 대한 크롤링이 완료되면 한꺼번에 오류가 난 부분들을 다시 크롤링
    3. 크롤링이 차단 당하거나 오류가 날시, continue 하여 다음 url로 바로 넘어가는게 아니라 n분 간격으로 n번까지 더 시도해서 scrape해보고 그럼에도 계속 오류가 발생하면 그 지점은 나중에 한꺼번에 재크롤링 할 수 있게 저장하고, 다음 url로 넘어가 이어서 scrape하도록 설계
    4. 최종적으로는, 로봇처럼 보이지 않도록, 위 3번 방법에 랜덤한 time.sleep을 추가하고 다양한 범위 시도 끝에 많이 느리지 않으면서도 차단을 당하지 않는 최적의 알고리즘 생성

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
import json
from datetime import datetime, timedelta

import time
import random
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

options = ChromeOptions()
options.add_argument('headless')
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36")
#path = '/Users/Python/chromedriver'
#driver = Chrome(executable_path=path, options=options)
driver = Chrome(options=options)

filename_contents = 'edited_content_2019.csv'
filename_errors = 'new_errors_2019.csv'
#content_list = []

for idx, url in enumerate(url_list):
    driver.get(url)
    time.sleep(random.uniform(8,20))
    req = driver.page_source
    soup = BeautifulSoup(req, 'lxml')
    try:
        # content 텍스트 데이터 수집
        content_tag = soup.select_one('div.rte__content')
        contents = content_tag.select('p')
        contents_collected = []
        for c in contents:
            content = c.get_text().strip()
            contents_collected.append(content) #한 프로젝트의 페이지 내 모든 설명/내용들을 하나로 모으고
        # risk and challenge 텍스트 데이터 수집
        try:
            risk_challenge_tag = soup.select_one('div#risksAndChallenges')
            risk_challenge_list = risk_challenge_tag.select('p.js-risks-text.text-preline')
            for rc in risk_challenge_list:
                risk_challenge = rc.get_text().strip()
        except:
            risk_challenge=" "
        
        # content_list.append([idx, contents_collected, risk_challenge]) # 크롤링 된 펀딩 내용(텍스트)도 변수에 저장 
        
        #오류 지점 이후 이어서 크롤링 할 때, index 순서 관계없이 일단 csv파일에 바로 이어서 한줄씩 append (이후 최종 취합 후 index 번호로 정렬)
        content_list_since_error = []
        content_list_since_error.append([idx, contents_collected, risk_challenge])
        pd.DataFrame(content_list_since_error, columns=['index','content','risk_challenge']).to_csv(filename_contents, index=False, header=False, mode='a', encoding='UTF-8')
        print(idx) #한 url에 대한 크롤링이 끝날 때마다 해당 index번호 출력
    
    # 오류 발생시 10번 재시도 
    except Exception as ex:
        print("Unexpected error at {}".format(idx), ex) #오류 지점 출력
        error_url = url
        count = 0
        while count < 11:
            print("Retrying:", count)
            driver.get(error_url)
            time.sleep(random.uniform(8,20))
            req = driver.page_source
            soup = BeautifulSoup(req, 'lxml')
            try:
                content_tag = soup.select_one('div.rte__content')
                contents = content_tag.select('p')
                contents_collected = []
                for c in contents:
                    content = c.get_text().strip()
                    contents_collected.append(content)
                
                try:
                    risk_challenge_tag = soup.select_one('div#risksAndChallenges')
                    risk_challenge_list = risk_challenge_tag.select('p.js-risks-text.text-preline')
                    for rc in risk_challenge_list:
                        risk_challenge = rc.get_text().strip()
                except:
                    risk_challenge=" "
                
                #오류 건에 대한 크롤링 재시도에서 크롤링 성공한 텍스트 데이터 csv에 추가/저장
                content_list_since_error = [] 
                content_list_since_error.append([idx, contents_collected, risk_challenge])
                pd.DataFrame(content_list_since_error, columns=['index','content','risk_challenge']).to_csv(filename_contents, index=False, header=False, mode='a', encoding='UTF-8')
                print(idx)
                break
                
            except Exception as et:
                print("Error while retrying:", et)
                time.sleep(random.uniform(100,130))
                count += 1
                continue
        
        #만약 재시도를 10번 했는데도 크롤링에 실패한다면, 원래 했던대로 오류 index, url, nan값을 csv파일들에 저장
        if count == 11:
            # content_list.append([idx, error_url, np.nan])
            
            # 오류 index, url 별도 오류 csv에 저장
            error_list = []
            error_list.append([idx, error_url])
            pd.DataFrame(error_list, columns=['index','error_url']).to_csv(filename_errors, index=False, header=False, mode='a', encoding='UTF-8')
            
            #오류난 지점에 대한 index, url도 일단은 텍스트 데이터 쌓고있는 csv 파일에 저장 (이후 재크롤링 후 대체)
            error_into_saved_csv = []
            error_into_saved_csv.append([idx, error_url, np.nan]) # risk_challenge 컬럼에 대한 값은 null로 지정
            pd.DataFrame(error_into_saved_csv, columns=['index','content','risk_challenge']).to_csv(filename_contents, index=False, header=False, mode='a', encoding='UTF-8')
        
        continue
    
driver.close()

In [ ]:
# Google Colab에서 Selenium 사용
# 재시도 횟수 --> 4번으로 변경

In [ ]:
!apt update
!apt install chromium-chromedriver
!pip install selenium

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
import json
from datetime import datetime, timedelta

import time
import random
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = Chrome('chromedriver', options=options)

#저장할 csv파일 이름/경로 지정 (Google Drive)
filename_contents = '/content/drive/My Drive/PlayData 빅데이터 분석 전문가 과정/Final_Project/Kickstarter_2019_12_12/content_crawled_2016.csv'
filename_errors = '/content/drive/My Drive/PlayData 빅데이터 분석 전문가 과정/Final_Project/Kickstarter_2019_12_12/errors_crawled_2016.csv'

for idx, url in enumerate(url_ks_2016[2165:], start=2165):
    driver.get(url)
    time.sleep(random.uniform(8,20))
    req = driver.page_source
    soup = BeautifulSoup(req, 'lxml')

    try:
        content_tag = soup.select_one('div.rte__content')
        contents = content_tag.select('p')
        contents_collected = []
        for c in contents:
            content = c.get_text().strip()
            contents_collected.append(content)

        try:
            risk_challenge_tag = soup.select_one('div#risksAndChallenges')
            risk_challenge_list = risk_challenge_tag.select('p.js-risks-text.text-preline')
            for rc in risk_challenge_list:
                risk_challenge = rc.get_text().strip()
        except:
            risk_challenge=" "
        
        content_list_since_error = []
        content_list_since_error.append([idx, contents_collected, risk_challenge])
        pd.DataFrame(content_list_since_error, columns=['index','content','risk_challenge']).to_csv(filename_contents, index=False, header=False, mode='a', encoding='UTF-8')
        print(idx)

    except Exception as ex:
        print("Unexpected error at {}".format(idx), ex)
        error_url = url
        count = 0
        while count < 5:
            print("Retrying:", count)
            driver.get(error_url)
            time.sleep(random.uniform(8,20))
            req = driver.page_source
            soup = BeautifulSoup(req, 'lxml')

            try:
                content_tag = soup.select_one('div.rte__content')
                contents = content_tag.select('p')
                contents_collected = []
                for c in contents:
                    content = c.get_text().strip()
                    contents_collected.append(content)

                try:
                    risk_challenge_tag = soup.select_one('div#risksAndChallenges')
                    risk_challenge_list = risk_challenge_tag.select('p.js-risks-text.text-preline')
                    for rc in risk_challenge_list:
                        risk_challenge = rc.get_text().strip()
                except:
                    risk_challenge=" "

                content_list_since_error = []
                content_list_since_error.append([idx, contents_collected, risk_challenge])
                pd.DataFrame(content_list_since_error, columns=['index','content','risk_challenge']).to_csv(filename_contents, index=False, header=False, mode='a', encoding='UTF-8')
                print("Retry successful")
                print(idx)
                break

            except Exception as et:
                print("Error while retrying:", et)
                time.sleep(random.uniform(100,130))
                count += 1
                continue

        if count == 5:
            error_into_saved_csv = []
            error_into_saved_csv.append([idx, np.nan, np.nan])
            pd.DataFrame(error_into_saved_csv, columns=['index','content','risk_challenge']).to_csv(filename_contents, index=False, header=False, mode='a', encoding='UTF-8')
            
            error_list = []
            error_list.append([idx, error_url])
            pd.DataFrame(error_list, columns=['index','error_url']).to_csv(filename_errors, index=False, header=False, mode='a', encoding='UTF-8')
            print("Retry failed")

        continue

driver.close()